In [1]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from joblib import load,dump

In [2]:
df= pd.read_csv('data/Hotel_review_feature_engineering.csv')
df.head()

,Unnamed: 0,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,...,wi_fi,air_conditioning,breakfast,booking_com,room_problem,location,staff,bed_and_room,month,year
0,0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Russia,I am so angry that i made this post available...,397,1403,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,8,2017
1,1,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Ireland,No Negative,0,1403,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,8,2017
2,2,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,Australia,Rooms are nice but for elderly a bit difficul...,42,1403,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,7,2017
3,3,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,UK,My room was dirty and I was afraid to walk ba...,210,1403,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,7,2017
4,4,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/24/2017,7.7,Hotel Arena,New Zealand,You When I booked with your company on line y...,140,1403,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7,2017


In [3]:
class Review_recommender():
    def __init__(self,data=df):
        self.data = data
        self.clean_df = self.data.groupby('Hotel_Name').agg({'Negative_Review':', '.join,'Positive_Review':', '.join}).reset_index()
        self.clean_df['Combined_Review'] = self.clean_df['Positive_Review'].astype(str) + self.clean_df['Negative_Review'].astype(str)
        self.clean_df["Combined_Review"] = self.clean_df["Combined_Review"].apply(lambda x: x.replace("No Negative", "").replace("No Positive", ""))
        self.clean_df['Combined_Review'] = self.clean_df['Combined_Review'].apply(lambda x: " ".join(x.lower() for x in x.split()))
        stop = stopwords.words('english')
        self.clean_df['Combined_Review'] = self.clean_df['Combined_Review'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
        tfv = TfidfVectorizer(min_df=3, max_features =None,strip_accents='unicode',analyzer='word',token_pattern=r'\w{1,}',ngram_range=(1,3))
        self.tfv_matrix = tfv.fit_transform(self.clean_df['Combined_Review'])
        self.clean_df.drop(['Combined_Review','Positive_Review','Negative_Review'],1,inplace=True)
        del tfv,self.data,stop
        self.cos_sim = linear_kernel(self.tfv_matrix, self.tfv_matrix)
        self.indices = pd.Series(self.clean_df.index, index=self.clean_df['Hotel_Name']).drop_duplicates()
        
    def recommend(self,index):
        index = self.indices[index]
        sim_scores = list(enumerate(self.cos_sim[index]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        sim_scores = sim_scores[1:11]
        hotel_indices = [i[0] for i in sim_scores]
        for i in self.clean_df['Hotel_Name'].iloc[hotel_indices]:
            print(i)

In [4]:
recommender = Review_recommender()
recommender.recommend('Hotel Arena')

Park Plaza London Riverbank
Hampshire Hotel Amsterdam American
Hilton London Metropole
Grand Royale London Hyde Park
The Student Hotel Amsterdam City
Park Plaza Vondelpark Amsterdam
NH Collection Amsterdam Grand Hotel Krasnapolsky
The Principal London
Radisson Blu Hotel Amsterdam
Holiday Inn London Kensington


In [5]:
dump(recommender,'model/recommender_based_on_reviews.joblib')

['model/recommender_based_on_reviews.joblib']

In [6]:
model = load('model/recommender_based_on_reviews.joblib')

In [7]:
model.recommend('Hotel Pulitzer')

Royal Passeig de Gracia
Hotel Espa a Ramblas
Avenida Palace
Catalonia Plaza Catalunya
Hotel Regina
Hotel Barcelona Catedral
Negresco Princess 4 Sup
Olivia Plaza Hotel
H10 Cubik 4 Sup
Petit Palace Boqueria Garden
